# Mehanizmi pažnje i transformeri

Jedan od glavnih nedostataka rekurentnih mreža je taj što sve riječi u nizu imaju isti utjecaj na rezultat. To uzrokuje suboptimalne performanse kod standardnih LSTM encoder-decoder modela za zadatke prevođenja sekvenci, poput prepoznavanja imenovanih entiteta i strojnog prevođenja. U stvarnosti, određene riječi u ulaznom nizu često imaju veći utjecaj na izlazne sekvence od drugih.

Razmotrimo model prevođenja sekvenci, poput strojnog prevođenja. On se implementira pomoću dvije rekurentne mreže, gdje jedna mreža (**encoder**) sažima ulazni niz u skriveno stanje, a druga mreža (**decoder**) razvija to skriveno stanje u prevedeni rezultat. Problem s ovim pristupom je što završno stanje mreže teško pamti početak rečenice, što uzrokuje lošu kvalitetu modela kod dugih rečenica.

**Mehanizmi pažnje** omogućuju ponderiranje kontekstualnog utjecaja svakog ulaznog vektora na svaku izlaznu predikciju RNN-a. To se implementira stvaranjem prečaca između međustanja ulaznog RNN-a i izlaznog RNN-a. Na taj način, prilikom generiranja izlaznog simbola $y_t$, uzimamo u obzir sva ulazna skrivena stanja $h_i$, s različitim težinskim koeficijentima $\alpha_{t,i}$.

![Slika koja prikazuje encoder/decoder model s aditivnim slojem pažnje](../../../../../translated_images/hr/encoder-decoder-attention.7a726296894fb567.webp)
*Encoder-decoder model s aditivnim mehanizmom pažnje u [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citirano iz [ovog blog posta](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matrica pažnje $\{\alpha_{i,j}\}$ predstavlja stupanj u kojem određene ulazne riječi sudjeluju u generiranju određene riječi u izlaznom nizu. Ispod je primjer takve matrice:

![Slika koja prikazuje uzorak poravnanja pronađenog pomoću RNNsearch-50, preuzeto iz Bahdanau - arviz.org](../../../../../translated_images/hr/bahdanau-fig3.09ba2d37f202a6af.webp)

*Slika preuzeta iz [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Slika 3)*

Mehanizmi pažnje odgovorni su za velik dio trenutnog ili gotovo trenutnog stanja umjetnosti u obradi prirodnog jezika. Međutim, dodavanje pažnje značajno povećava broj parametara modela, što je dovelo do problema skaliranja s RNN-ovima. Ključno ograničenje skaliranja RNN-ova je to što rekurentna priroda modela otežava grupiranje i paralelizaciju treninga. U RNN-u svaki element sekvence mora se obraditi redoslijedom, što znači da se ne može lako paralelizirati.

Usvajanje mehanizama pažnje u kombinaciji s ovim ograničenjem dovelo je do stvaranja sadašnjih transformera, koji predstavljaju stanje umjetnosti, a koje danas poznajemo i koristimo, od BERT-a do OpenGPT3.

## Transformer modeli

Umjesto prosljeđivanja konteksta svake prethodne predikcije u sljedeći korak evaluacije, **transformer modeli** koriste **pozicijske kodove** i **pažnju** kako bi uhvatili kontekst danog ulaza unutar zadanog prozora teksta. Slika ispod prikazuje kako pozicijski kodovi s pažnjom mogu uhvatiti kontekst unutar zadanog prozora.

![Animirani GIF koji prikazuje kako se evaluacije provode u transformer modelima.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Budući da se svaka ulazna pozicija neovisno mapira na svaku izlaznu poziciju, transformeri mogu bolje paralelizirati od RNN-ova, što omogućuje mnogo veće i izražajnije jezične modele. Svaka glava pažnje može se koristiti za učenje različitih odnosa između riječi, što poboljšava zadatke obrade prirodnog jezika.

## Izgradnja jednostavnog transformer modela

Keras ne sadrži ugrađeni sloj za transformere, ali možemo izgraditi vlastiti. Kao i prije, fokusirat ćemo se na klasifikaciju teksta AG News skupa podataka, no vrijedi napomenuti da transformer modeli pokazuju najbolje rezultate na složenijim zadacima obrade prirodnog jezika.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Novi slojevi u Kerasi trebaju naslijediti klasu `Layer` i implementirati metodu `call`. Počnimo s **Positional Embedding** slojem. Koristit ćemo [neki kod iz službene Keras dokumentacije](https://keras.io/examples/nlp/text_classification_with_transformer/). Pretpostavit ćemo da sve ulazne sekvence popunjavamo do duljine `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Ovaj sloj sastoji se od dva sloja `Embedding`: za ugradnju tokena (na način koji smo ranije raspravili) i pozicija tokena. Pozicije tokena stvaraju se kao niz prirodnih brojeva od 0 do `maxlen` koristeći `tf.range`, a zatim se prosljeđuju kroz sloj za ugradnju. Dva dobivena vektora ugradnje se zatim zbrajaju, stvarajući pozicijski ugrađenu reprezentaciju ulaza oblika `maxlen`$\times$`embed_dim`.

Sada, implementirajmo transformer blok. On će uzeti izlaz prethodno definiranog sloja za ugradnju:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Transformer primjenjuje `MultiHeadAttention` na ulaz s pozicijskim kodiranjem kako bi proizveo vektor pažnje dimenzije `maxlen`$\times$`embed_dim`, koji se zatim miješa s ulazom i normalizira pomoću `LayerNormalization`.

> **Napomena**: `LayerNormalization` je sličan `BatchNormalization` koji je obrađen u dijelu *Računalni vid* ovog programa učenja, ali normalizira izlaze prethodnog sloja za svaki uzorak u treningu neovisno, kako bi ih doveo u raspon [-1..1].

Izlaz ovog sloja zatim se prosljeđuje kroz `Dense` mrežu (u našem slučaju - perceptron s dva sloja), a rezultat se dodaje konačnom izlazu (koji se ponovno podvrgava normalizaciji).

Sada smo spremni definirati kompletan model transformatora:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer modeli

**BERT** (Bidirectional Encoder Representations from Transformers) je vrlo velika višeslojna transformacijska mreža s 12 slojeva za *BERT-base* i 24 za *BERT-large*. Model se prvo unaprijed trenira na velikom korpusu tekstualnih podataka (WikiPedia + knjige) koristeći nenadzirano učenje (predviđanje maskiranih riječi u rečenici). Tijekom unaprijednog treniranja model usvaja značajnu razinu razumijevanja jezika, što se kasnije može iskoristiti s drugim skupovima podataka putem finog podešavanja. Ovaj proces naziva se **transferno učenje**.

![slika s http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/hr/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Postoji mnogo varijacija transformacijskih arhitektura, uključujući BERT, DistilBERT, BigBird, OpenGPT3 i druge, koje se mogu fino podešavati.

Pogledajmo kako možemo koristiti unaprijed trenirani BERT model za rješavanje našeg tradicionalnog problema klasifikacije sekvenci. Posudit ćemo ideju i dio koda iz [službene dokumentacije](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Za učitavanje unaprijed treniranih modela koristit ćemo **Tensorflow hub**. Prvo, učitajmo BERT-specifični vektorizator:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Važno je koristiti isti vektorizator koji je korišten za treniranje originalne mreže. Također, BERT vektorizator vraća tri komponente:
* `input_word_ids`, što je niz brojeva tokena za ulaznu rečenicu
* `input_mask`, koji pokazuje koji dio niza sadrži stvarni unos, a koji je popuna. Slično je maski koju proizvodi sloj `Masking`
* `input_type_ids` koristi se za zadatke modeliranja jezika i omogućuje specificiranje dvije ulazne rečenice u jednom nizu.

Zatim možemo instancirati BERT ekstraktor značajki:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Dakle, BERT sloj vraća nekoliko korisnih rezultata:
* `pooled_output` je rezultat prosjeka svih tokena u sekvenci. Možete ga smatrati inteligentnim semantičkim ugrađivanjem cijele mreže. Ekvivalentan je izlazu sloja `GlobalAveragePooling1D` u našem prethodnom modelu.
* `sequence_output` je izlaz posljednjeg transformacijskog sloja (odgovara izlazu `TransformerBlock` u našem modelu iznad).
* `encoder_outputs` su izlazi svih transformacijskih slojeva. Budući da smo učitali BERT model s 4 sloja (kao što vjerojatno možete zaključiti iz imena koje sadrži `4_H`), ima 4 tenzora. Posljednji je isti kao `sequence_output`.

Sada ćemo definirati end-to-end model za klasifikaciju. Koristit ćemo *funkcionalnu definiciju modela*, gdje definiramo ulaz modela, a zatim pružamo niz izraza za izračun njegovog izlaza. Također ćemo postaviti težine BERT modela kao ne-trenirajuće i trenirati samo završni klasifikator:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Unatoč tome što postoji malo parametara za treniranje, proces je prilično spor jer je BERT-ov ekstraktor značajki računalno zahtjevan. Čini se da nismo uspjeli postići zadovoljavajuću točnost, bilo zbog nedostatka treninga ili nedostatka parametara modela.

Pokušajmo otključati težine BERT-a i trenirati ga također. Ovo zahtijeva vrlo malu stopu učenja, kao i pažljiviju strategiju treniranja s **zagrijavanjem** koristeći **AdamW** optimizator. Koristit ćemo paket `tf-models-official` za kreiranje optimizatora:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Kao što možete vidjeti, treniranje ide prilično sporo - ali možda biste željeli eksperimentirati i trenirati model kroz nekoliko epoha (5-10) kako biste vidjeli možete li postići najbolji rezultat u usporedbi s pristupima koje smo koristili ranije.

## Huggingface Transformers knjižnica

Još jedan vrlo čest (i malo jednostavniji) način korištenja Transformer modela je [HuggingFace paket](https://github.com/huggingface/), koji pruža jednostavne građevne blokove za različite NLP zadatke. Dostupan je i za Tensorflow i za PyTorch, još jedan vrlo popularan okvir za neuronske mreže.

> **Napomena**: Ako vas ne zanima kako radi Transformers knjižnica - možete preskočiti na kraj ovog bilježnika, jer nećete vidjeti ništa bitno drugačije od onoga što smo već radili. Ponavljat ćemo iste korake treniranja BERT modela koristeći drugu knjižnicu i znatno veći model. Stoga proces uključuje prilično dugo treniranje, pa možda želite samo pregledati kod.

Pogledajmo kako se naš problem može riješiti koristeći [Huggingface Transformers](http://huggingface.co).


Prvo što trebamo učiniti je odabrati model koji ćemo koristiti. Osim nekih ugrađenih modela, Huggingface sadrži [online repozitorij modela](https://huggingface.co/models), gdje možete pronaći mnogo više unaprijed treniranih modela koje je kreirala zajednica. Svi ti modeli mogu se učitati i koristiti jednostavnim navođenjem imena modela. Svi potrebni binarni datoteke za model automatski će se preuzeti.

Ponekad ćete trebati učitati vlastite modele, u tom slučaju možete navesti direktorij koji sadrži sve relevantne datoteke, uključujući parametre za tokenizer, datoteku `config.json` s parametrima modela, binarne težine itd.

Na temelju imena modela, možemo instancirati i model i tokenizer. Započnimo s tokenizerom:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Objekt `tokenizer` sadrži funkciju `encode` koja se može izravno koristiti za kodiranje teksta:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Također možemo koristiti tokenizer za kodiranje sekvence na način prikladan za prosljeđivanje modelu, tj. uključujući polja `token_ids`, `input_mask`, itd. Također možemo specificirati da želimo Tensorflow tenzore pružanjem argumenta `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

U ovom slučaju, koristit ćemo unaprijed istrenirani BERT model nazvan `bert-base-uncased`. *Uncased* označava da model nije osjetljiv na velika i mala slova.

Prilikom treniranja modela, potrebno je osigurati tokenizirani niz kao ulaz, i stoga ćemo osmisliti procesnu liniju za obradu podataka. Budući da je `tokenizer.encode` Python funkcija, koristit ćemo isti pristup kao u posljednjoj jedinici pozivajući je pomoću `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Sada možemo učitati stvarni model koristeći paket `BertForSequenceClassification`. Ovo osigurava da naš model već ima potrebnu arhitekturu za klasifikaciju, uključujući završni klasifikator. Vidjet ćete poruku upozorenja koja navodi da težine završnog klasifikatora nisu inicijalizirane i da bi model zahtijevao prethodnu obuku - to je potpuno u redu, jer upravo to namjeravamo učiniti!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Kao što možete vidjeti iz `summary()`, model sadrži gotovo 110 milijuna parametara! Vjerojatno, ako želimo jednostavan zadatak klasifikacije na relativno malom skupu podataka, ne želimo trenirati osnovni BERT sloj:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Sada smo spremni započeti treniranje!

> **Napomena**: Treniranje BERT modela u punom opsegu može biti vrlo dugotrajno! Stoga ćemo ga trenirati samo za prvih 32 serije. Ovo je samo da pokažemo kako se postavlja treniranje modela. Ako ste zainteresirani za treniranje u punom opsegu - samo uklonite parametre `steps_per_epoch` i `validation_steps`, i pripremite se na čekanje!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Ako povećate broj iteracija i pričekate dovoljno dugo, te trenirate kroz nekoliko epoha, možete očekivati da će BERT klasifikacija pružiti najbolju točnost! To je zato što BERT već prilično dobro razumije strukturu jezika, pa je potrebno samo fino prilagoditi završni klasifikator. Međutim, budući da je BERT veliki model, cijeli proces treniranja traje dugo i zahtijeva ozbiljnu računalnu snagu! (GPU, i po mogućnosti više od jednog).

> **Napomena:** U našem primjeru koristili smo jedan od najmanjih unaprijed istreniranih BERT modela. Postoje veći modeli koji vjerojatno daju bolje rezultate.


## Ključne točke

U ovoj jedinici smo se upoznali s vrlo recentnim arhitekturama modela temeljenim na **transformerima**. Primijenili smo ih za naš zadatak klasifikacije teksta, ali na sličan način BERT modeli mogu se koristiti za ekstrakciju entiteta, odgovaranje na pitanja i druge NLP zadatke.

Transformerski modeli predstavljaju trenutno najnaprednije stanje u NLP-u, i u većini slučajeva trebali bi biti prvo rješenje s kojim započinjete eksperimentiranje prilikom implementacije prilagođenih NLP rješenja. Međutim, razumijevanje osnovnih principa rekurentnih neuronskih mreža, o kojima smo raspravljali u ovom modulu, izuzetno je važno ako želite izgraditi napredne neuronske modele.



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije koje proizlaze iz korištenja ovog prijevoda.
